# V-Column Migration Using Resource-Specific Migration Tools

This notebook demonstrates how to use the new resource-specific migration managers to perform V-column migrations efficiently. The new approach uses `client.accounts.create_migration_manager()` which automatically handles account-specific requirements like the `user_type` parameter.

## Key Benefits:
- **Resource-Aware**: Automatically handles resource-specific parameters (like `user_type` for accounts)
- **Simplified Code**: ~10 lines vs ~200 lines of manual code
- **Better Error Handling**: Comprehensive error tracking and reporting
- **Batch Processing**: Efficient processing with configurable batch sizes
- **Conflict Detection**: Advanced analysis before migration
- **Validation**: Built-in field validation
- **Parallel Processing**: Multi-threaded execution for better performance
- **Dynamic**: Works with any resource type without hardcoded assumptions

In [1]:
# Setup - Initialize client and migration manager
import os
from neon_crm import NeonClient, Role, UserType
from neon_crm.migration_tools import MigrationStrategy

# Initialize client with built-in retry logic
client = NeonClient(
    default_role=Role.EDITOR
)

# Initialize migration manager for accounts with user_type specified
migration_manager = client.accounts.create_migration_manager(user_type=UserType.INDIVIDUAL)
print("✓ Migration manager initialized")

✓ Migration manager initialized


In [2]:
# Define the same mapping as the original notebook
mapping = {
    'V-Canvassing': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Canvassing/literature Drop'},
    'V-Communications Team - website, newsletter, blog, messaging, graphic design': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Graphic Design'},
    'V-Data Entry ': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Data Entry'},
    'V-Diversity Outreach Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Diversity Outreach'},
    'V-Driver assisting Canvassers/Voters': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Driver to assist canvassers/voters'},
    'V-Events Planning - help organize/coordinate booths/tables': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Staff/assign/engage at events'},
    'V-Food Team- for events, meetings, during election season': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Food team for canvassers and/or candidate events'},
    'V-Front_Desk Reception Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Office/front desk help'},
    'V-Get out the Vote Event Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Voter registration'},
    'V-Graphic Design': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Graphic Design'},
    'V-Help with Mailings': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Mailings'},
    'V-Large Signs-Help place large signs': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Install large signs'},
    'V-Letter writing/ articles/research': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Writing'},
    'V-Moving Furniture': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Carry/load event materials'},
    'V-Office Handy Work': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Fixing & odd jobs'},
    'V-Office materials/supplies organization': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Organize spaces & materials'},
    'V-Phone/Text Banking': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Make Phone Calls'},
    'V-Poll Watcher or Worker': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Poll worker or observer'},
    'V-Postcard Campaigns writing': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Postcard writing from provided script'},
    'V-Rallies Support - hold signs, flags': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Staff/assign/engage at events'},
    'V-Rural Outreach Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Rural outreach'},
    'V-Sign making for Rallies': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Make Handmade Rally Signs'},
    'V-Signs-Making (Hand lettered)': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Make Handmade Rally Signs'},
    'V-Social Media Team': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Social Media'},
    'V-Staffing Booths/tables at events': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Staff/assign/engage at events'},
    'V-State Team Housing - during elections': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Provide temporary housing for campaign staff'},
    'V-Tech Team': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Technology'},
    'V-Tech Team Interests': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Technology'},
    'V-Volunteer Campaign & Election Activities': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164},
    'V-Volunteer Form Other Comments': {'field': 'V-Volunteer Form Other Comments', 'field_id': 166},
    'V-Volunteer How did you hear about us': {'field': 'V-Volunteer How did you hear about us', 'field_id': 165},
    'V-Volunteer Interests': {'field': 'V-Volunteer Interests', 'field_id': 160},
    'V-Volunteer Outreach Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Diversity Outreach'},
    'V-Volunteer Skills': {'field': 'V-Volunteer Skills', 'field_id': 163},
    'V-Voter Registration - Refer to CV Votes': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Voter registration'},
    'V-Want a Yard Sign': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Assemble and/or place yard signs'},
}

print(f"✓ Mapping loaded with {len(mapping)} field mappings")
print(f"✓ Skipping {len([k for k, v in mapping.items() if v == 'TODO'])} TODO mappings")

✓ Mapping loaded with 36 field mappings
✓ Skipping 0 TODO mappings


In [3]:
# Step 1: Create migration plan from the field mapping
migration_plan = migration_manager.create_migration_plan_from_mapping(mapping)

print(f"✓ Migration plan created with {len(migration_plan.mappings)} mappings")
print(f"✓ Plan settings: batch_size={migration_plan.batch_size}, dry_run={migration_plan.dry_run}")

# Show first few mappings as examples
print("\n📋 First 3 migration mappings:")
for i, migration_mapping in enumerate(migration_plan.mappings[:3]):
    print(f"  {i+1}. {migration_mapping.source_field} -> {migration_mapping.target_field} ({migration_mapping.strategy.value})")

✓ Migration plan created with 36 mappings
✓ Plan settings: batch_size=100, dry_run=True

📋 First 3 migration mappings:
  1. V-Canvassing -> V-Volunteer Campaign & Election Activities (add_option)
  2. V-Communications Team - website, newsletter, blog, messaging, graphic design -> V-Volunteer Skills (add_option)
  3. V-Data Entry  -> V-Volunteer Skills (add_option)


In [4]:
# Step 2: Analyze potential conflicts before migration
print("🔍 Analyzing potential migration conflicts...")
conflict_report = migration_manager.analyze_migration_conflicts(migration_plan)

print(f"\n📊 Conflict Analysis Results:")
print(f"  • Missing source fields: {len(conflict_report.field_conflicts.get('missing_source', []))}")
print(f"  • Missing target fields: {len(conflict_report.field_conflicts.get('missing_target', []))}")
print(f"  • Type conflicts: {len(conflict_report.type_conflicts)}")
print(f"  • Value conflicts: {len(conflict_report.value_conflicts)}")
print(f"  • Suggestions: {len(conflict_report.resolution_suggestions)}")

if conflict_report.field_conflicts:
    print("\n⚠️  Field Conflicts:")
    for conflict_type, fields in conflict_report.field_conflicts.items():
        print(f"  {conflict_type}: {fields[:3]}{'...' if len(fields) > 3 else ''}")

if conflict_report.resolution_suggestions:
    print("\n💡 Suggestions:")
    for suggestion in conflict_report.resolution_suggestions[:3]:
        print(f"  • {suggestion}")

🔍 Analyzing potential migration conflicts...

📊 Conflict Analysis Results:
  • Missing source fields: 0
  • Missing target fields: 0
  • Type conflicts: 30
  • Value conflicts: 0
  • Suggestions: 0


In [5]:
# Step 3a: CLEANUP-ONLY for already migrated account
# Since you already migrated the fields, this will ONLY clear source fields without re-migrating
specific_accounts = [6488]  # Your test account

print(f"🧹 Creating CLEANUP-ONLY plan for account: {specific_accounts}")
print("📌 This will ONLY clear source fields - no migration will happen")

# Create a cleanup-only plan (perfect for post-migration cleanup)
cleanup_only_plan = migration_manager.create_cleanup_only_plan(
    field_mapping=mapping, 
    resource_ids=specific_accounts,
    dry_run=True  # Test first!
)

print(f"✓ Cleanup-only plan created with {len(cleanup_only_plan.mappings)} mappings")
print(f"✓ Will process account: {cleanup_only_plan.resource_ids}")
print(f"✓ Cleanup mode: {cleanup_only_plan.cleanup_only}")
print(f"✓ Will clear source fields: {not cleanup_only_plan.mappings[0].preserve_source}")

# Execute cleanup-only (dry run first)
print("\n🧪 Executing CLEANUP-ONLY (DRY RUN)...")
cleanup_results = migration_manager.execute_migration_plan(cleanup_only_plan)

print(f"\n📈 Cleanup-Only Results (DRY RUN):")
print(f"  • Total resources processed: {cleanup_results.total_resources}")
print(f"  • Source fields to clear: {cleanup_results.successful_migrations}")
print(f"  • Failed clears: {cleanup_results.failed_migrations}")
print(f"  • Skipped (no data): {cleanup_results.skipped_migrations}")

if cleanup_results.errors:
    print("\n❌ Errors in cleanup:")
    for error in cleanup_results.errors:
        print(f"  • {error}")

print("\n✅ Dry run completed - check logs for '[DRY RUN] Would clear source field' messages")

# 🚀 EXECUTE CLEANUP FOR REAL (uncomment next 4 lines):
cleanup_only_plan.dry_run = False
print("⚠️  EXECUTING REAL CLEANUP - This will clear source fields only!")
real_cleanup_results = migration_manager.execute_migration_plan(cleanup_only_plan)
print(f"🎉 Cleanup complete: {real_cleanup_results.successful_migrations} fields cleared")

🧹 Creating CLEANUP-ONLY plan for account: [6488]
📌 This will ONLY clear source fields - no migration will happen
✓ Cleanup-only plan created with 36 mappings
✓ Will process account: [6488]
✓ Cleanup mode: True
✓ Will clear source fields: True

🧪 Executing CLEANUP-ONLY (DRY RUN)...

📈 Cleanup-Only Results (DRY RUN):
  • Total resources processed: 1
  • Source fields to clear: 33
  • Failed clears: 0
  • Skipped (no data): 0

✅ Dry run completed - check logs for '[DRY RUN] Would clear source field' messages
⚠️  EXECUTING REAL CLEANUP - This will clear source fields only!


Failed to clear source field 'V-Volunteer Form Other Comments' for resource 6488
Failed to clear source field 'V-Volunteer How did you hear about us' for resource 6488


🎉 Cleanup complete: 31 fields cleared


In [ ]:
# Step 3b: SMART MIGRATION for already migrated account
# This checks if target fields already have the expected values and only clears source fields
specific_accounts = [6488]  # Your test account

print(f"🧠 Creating SMART MIGRATION plan for account: {specific_accounts}")
print("📌 This will check target values and only migrate if needed, then clear source fields")

# Create a smart migration plan (checks target status before migrating)
smart_migration_plan = migration_manager.create_smart_migration_plan(
    field_mapping=mapping, 
    resource_ids=specific_accounts,
    dry_run=True,  # Test first!
    clear_source_fields=True  # Clear source fields after checking/migrating
)

print(f"✓ Smart migration plan created with {len(smart_migration_plan.mappings)} mappings")
print(f"✓ Will process account: {smart_migration_plan.resource_ids}")
print(f"✓ Smart mode: {smart_migration_plan.smart_migration}")
print(f"✓ Will clear source fields: {not smart_migration_plan.mappings[0].preserve_source}")

# Execute smart migration (dry run first)
print("\n🧪 Executing SMART MIGRATION (DRY RUN)...")
smart_results = migration_manager.execute_migration_plan(smart_migration_plan)

print(f"\n📈 Smart Migration Results (DRY RUN):")
print(f"  • Total resources processed: {smart_results.total_resources}")
print(f"  • Successful operations: {smart_results.successful_migrations}")
print(f"  • Failed operations: {smart_results.failed_migrations}")
print(f"  • Skipped (no data): {smart_results.skipped_migrations}")

if smart_results.errors:
    print("\n❌ Errors in smart migration:")
    for error in smart_results.errors:
        print(f"  • {error}")

print("\n✅ Dry run completed - check logs for target status checking")

# 🚀 EXECUTE SMART MIGRATION FOR REAL (uncomment next 4 lines):
smart_migration_plan.dry_run = False
print("⚠️  EXECUTING REAL SMART MIGRATION - This will check targets and clear source fields!")
real_smart_results = migration_manager.execute_migration_plan(smart_migration_plan)
print(f"🎉 Smart migration complete: {real_smart_results.successful_migrations} operations successful")

🧠 Creating SMART MIGRATION plan for account: [6488]
📌 This will check target values and only migrate if needed, then clear source fields
✓ Smart migration plan created with 36 mappings
✓ Will process account: [6488]
✓ Smart mode: True
✓ Will clear source fields: True

🧪 Executing SMART MIGRATION (DRY RUN)...

📈 Smart Migration Results (DRY RUN):
  • Total resources processed: 1
  • Successful operations: 2
  • Failed operations: 0
  • Skipped (no data): 0

✅ Dry run completed - check logs for target status checking
⚠️  EXECUTING REAL SMART MIGRATION - This will check targets and clear source fields!


In [ ]:
# Step 3a: Test migration on specific accounts (NEW FEATURE!)
# This is useful for testing or doing partial migrations
specific_accounts = [6488]  # Replace with actual account IDs

print(f"🎯 Creating migration plan for specific accounts: {specific_accounts}")

# Create a migration plan that only processes the specified accounts
targeted_migration_plan = migration_manager.create_migration_plan_for_resources(
    field_mapping=mapping, 
    resource_ids=specific_accounts,
    dry_run=False
)

print(f"✓ Targeted migration plan created with {len(targeted_migration_plan.mappings)} mappings")
print(f"✓ Will only process accounts: {targeted_migration_plan.resource_ids}")

# Execute targeted migration
print("\n🧪 Executing targeted migration...")
targeted_results = migration_manager.execute_migration_plan(targeted_migration_plan)

print(f"\n📈 Targeted Migration Results:")
print(f"  • Total resources processed: {targeted_results.total_resources}")
print(f"  • Successful migrations: {targeted_results.successful_migrations}")
print(f"  • Failed migrations: {targeted_results.failed_migrations}")
print(f"  • Skipped (no data): {targeted_results.skipped_migrations}")

if targeted_results.errors:
    print("\n❌ Errors in targeted migration:")
    for error in targeted_results.errors:
        print(f"  • {error}")
else:
    print("\n✅ Targeted migration completed successfully!")

In [ ]:
# Step 5: Execute REAL migration with source field cleanup (uncomment when ready)
# CAUTION: This will make actual changes AND clear source fields in your Neon CRM data!

# FOR SINGLE TEST ACCOUNT (uncomment to execute):
# cleanup_migration_plan.dry_run = False
# print("⚠️  EXECUTING REAL MIGRATION WITH CLEANUP - This will modify and clear fields!")
# real_cleanup_results = migration_manager.execute_migration_plan(cleanup_migration_plan)
# print(f"🎉 Migration with cleanup complete: {real_cleanup_results.successful_migrations} successful")

# FOR ALL ACCOUNTS (FUTURE - currently commented out):
# # Create migration plan for ALL accounts with source field cleanup
# # all_accounts_cleanup_plan = migration_manager.create_migration_plan_with_cleanup(
# #     field_mapping=mapping,
# #     resource_ids=None,  # None means all accounts
# #     dry_run=False
# # )
# # 
# # print("⚠️⚠️ EXECUTING FULL MIGRATION WITH CLEANUP - This will modify ALL accounts!")
# # all_results = migration_manager.execute_migration_plan(all_accounts_cleanup_plan)
# # print(f"🎉 Full migration complete: {all_results.successful_migrations} successful")

print("💡 Uncomment the appropriate section above when ready to execute!")

In [ ]:
# Step 5: Execute REAL migration (uncomment when ready)
# CAUTION: This will make actual changes to your Neon CRM data!

# # Set dry_run to False for real execution
# migration_plan.dry_run = False
# 
# print("⚠️  EXECUTING REAL MIGRATION - This will modify your Neon CRM data!")
# 
# # Execute the real migration
# real_results = migration_manager.execute_migration_plan(migration_plan)
# 
# print(f"\n🎉 Real Migration Results:")
# print(f"  • Total resources: {real_results.total_resources}")
# print(f"  • Successful migrations: {real_results.successful_migrations}")
# print(f"  • Failed migrations: {real_results.failed_migrations}")
# print(f"  • Total errors: {len(real_results.errors)}")
# 
# if real_results.errors:
#     print("\n❌ Errors during migration:")
#     for error in real_results.errors:
#         print(f"  • {error}")

print("💡 Uncomment the code above when ready to execute the real migration!")

## 🎉 Migration Complete!

### Summary of what we accomplished:

1. **✅ Resource-Specific Managers**: Migration tooling now lives on each resource (e.g., `client.accounts.create_migration_manager()`)
2. **✅ Dynamic Parameter Handling**: Automatically handles resource-specific requirements like `user_type` for accounts
3. **✅ Simplified Code**: Replaced ~200 lines of manual migration code with ~10 lines using the migration tools
4. **✅ Better Error Handling**: Comprehensive error tracking and detailed reporting
5. **✅ Conflict Detection**: Advanced analysis to identify potential issues before migration
6. **✅ Batch Processing**: Efficient processing with configurable batch sizes
7. **✅ Validation**: Built-in field validation ensures data integrity
8. **✅ Dry Run Testing**: Safe testing without making actual changes
9. **✅ Detailed Reporting**: Comprehensive statistics and per-mapping results

### Key improvements over manual approach:
- **Resource-Aware**: Each resource type handles its own specific requirements
- **Type-Safe**: No more generic parameter guessing - each resource knows what it needs
- **Single Responsibility**: Each function has one clear purpose
- **Parallel Processing**: Multi-threaded execution for better performance
- **Robust Error Handling**: Detailed error messages with context
- **Strategy Support**: Multiple migration strategies (REPLACE, MERGE, COPY_IF_EMPTY, ADD_OPTION)
- **Maintainable**: Much easier to understand, test, and modify
- **Scalable**: Easy to add migration support to other resource types

### Usage for different resources:
```python
# For accounts (handles user_type automatically)
accounts_migrator = client.accounts.create_migration_manager(user_type="INDIVIDUAL")

# For events (would handle event-specific parameters)
# events_migrator = client.events.create_migration_manager(start_date="2024-01-01")

# For donations (no special parameters needed)
# donations_migrator = client.donations.create_migration_manager()
```

The resource-specific migration tools provide a much more robust, maintainable, and dynamic solution for field migrations!